<a href="https://colab.research.google.com/github/joaohenrifranco/bankruptcy-prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: ignored

In [ ]:
path = '/content/gdrive/MyDrive/Inteligência Computacional/data.csv'

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.isna().sum().sort_values()

In [ ]:
df.describe()

In [ ]:
df['Bankrupt?'].value_counts()

In [ ]:
df.dtypes

In [ ]:
import seaborn as sns

corr = df.corr()

cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
df.hist(bins=200, figsize=(30, 30))

In [ ]:
pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(100, 100))

In [ ]:
pd.plotting.scatter_matrix(df, alpha=0.2)